In [1]:
!pip install scipy==1.10.1 gensim nltk pyldavis


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary
from gensim import models
import glob
import nltk
from nltk.stem import *
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

In [3]:
nltk.download('stopwords')
stopwords = set(nltk.corpus.stopwords.words('english'))

nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/rmartins/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/rmartins/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [21]:
custom_stopwords = ['one', 'two', 'also', 'c', 'however']

aux_texts = []

# The two batches are slightly different
for filename in glob.glob('txts-webofscience-643-2177/*.txt'):
    with open(filename, encoding='utf16') as file:
        # Ignore first line
        text = file.readline()
        text = file.readline().split('\t')[4].strip()
        aux_texts.append(text)

for filename in glob.glob('txts-webofscience-*/*.txt'):
    with open(filename, encoding='utf16') as file:
        text = file.readline().split('\t')[4].strip()
        aux_texts.append(text)

stemmer = PorterStemmer()

stopwords.update(custom_stopwords)

texts = []

for text in aux_texts:
    words = set(word.lower() for word in nltk.word_tokenize(text) if word.isalpha())
    words = [word for word in words if word not in stopwords]
    #words = [stemmer.stem(word) for word in words if word not in stopwords]        

    # This is arbitrary; trying to remove noise and errors
    if len(words) > 10:
        texts.append(words)
        #print(words)

In [22]:
len(texts)

1693

In [28]:
our_dict = Dictionary(texts)
our_corpus = [our_dict.doc2bow(text) for text in texts]

In [29]:
%%time
# Train the model on the corpus.
our_lda_model = models.ldamodel.LdaModel(our_corpus, num_topics=10)

CPU times: user 1.8 s, sys: 0 ns, total: 1.8 s
Wall time: 1.8 s


In [30]:
vis_data = gensimvis.prepare(our_lda_model, our_corpus, our_dict)
pyLDAvis.display(vis_data)